# DeepTurkish Own Voice Testing

This notebook is loading a trained model and directly testing it on user provided audio recordings.

In [ ]:
import os

import torch
import torch.nn.functional as F
import librosa
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import utilities.utilities as utils
from model.neural_network import make_model
from model.data_loader import make_loaders
from decoders import decoders
from evaluation import test

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

## 1) Load the State Dicts,

In [ ]:
project_name ="METUbet+Mozilla"
model_name = '01_03__05_02_2021'

iter_meter = utils.IterMeter(project_name, model_name)

model_path = os.path.join("data","models and losses",project_name,model_name,"{}.pt".format(model_name))
model_state = torch.load(model_path) 

hyperparameters = model_state['hyperparameters']
data_parameters = model_state['data_parameters']

hyperparameters['model_dir'] = model_path # for loading the model

## 2) Make the Model, Criterion and the Loader

In [ ]:
# If you trained the model in Colab, you have to change a few paths
data_parameters['dataframe_dir_train'] = os.path.join("data","datasets","METUbet","data","METUbet_ordered_train.pkl")
data_parameters['dataframe_dir_test'] = os.path.join("data","datasets","METUbet","data","METUbet_ordered_test.pkl")
data_parameters['train_dir'] = os.path.join("data","datasets","METUbet","data","spectrograms","dB")
data_parameters['test_dir'] = os.path.join("data","datasets","METUbet","data","spectrograms","dB")

In [ ]:
# Create the test loader
_,_,test_loader = make_loaders(data_parameters, sortagrad=False)

# Create the model and the Criterion
model, _, _, _ = make_model(hyperparameters, data_parameters['blank'], len(test_loader), device)

## 3) Choose a Decoder

Choose a decoder for decoding the ctc output matrix.

In [ ]:
# Argmax decoder
decoder = decoders.Argmax_decoder(data_parameters['alphabet'], data_parameters['blank'])

In [ ]:
# BeamSearch Decoder
LM_text_name="NN_datasets_sentences"
beam_width = 5
prune_threshold = -7 # = log(0.001)

decoder = decoders.BeamSearch_decoder(data_parameters['alphabet'], data_parameters['blank'], beam_width, prune_threshold, LM_text_name)

In [ ]:
# LexiconSearch
tolerance = 1

# choose an apprroximator for the Lexicon Search algorithm
BW = 5
prune = -7 # = log(0.001)
LM_text_name="NN_datasets_sentences"

approximator_properties = ('BeamSearch+LM',data_parameters['blank'], BW, prune, LM_text_name)

decoder = decoders.LexiconSearch_decoder(data_parameters['alphabet'], tolerance, LM_text_name, approximator_properties)

## 4) Read an Audio CLip

In [ ]:
clip_dir = os.path.join("data","audio_clips")

audio_path = os.path.join(clip_dir, "elif_ses.wav")
#audio_path = os.path.join(clip_dir, "s1002-003.wav")

audio,sr = librosa.load(audio_path,sr=16000)
 
amplitude_spectrogram = np.abs(librosa.stft(audio,n_fft=512,hop_length=int(512/3),win_length=512))
dB_spectrogram = librosa.amplitude_to_db(amplitude_spectrogram,np.max(amplitude_spectrogram))

data = MinMaxScaler(copy=False).fit_transform(dB_spectrogram)

X = torch.from_numpy(data).to(device) # Convert to tensor 
X = X.unsqueeze(dim=0).unsqueeze(dim=0)

In [ ]:
output = model(X)
log_probs = F.log_softmax(output, dim=-1).detach().cpu().numpy()

decoder.decode(log_probs)